# Configurations

In [3]:
from IPython.display import display, HTML
import numpy as np
import pandas as pd

Test_file     = './Dataset/test_66516Ee.csv'
Test_file     = './Dataset/tst_renewal_predicted.csv'

# Test_dtype = {'id':np.str, 'perc_premium_paid_by_cash_credit':np.float32, 
# 'age_in_days':np.float32, 
# 'Income':np.float32,
# 'Count_3-6_months_late':np.float32, 
# 'Count_6-12_months_late':np.float32,
# 'Count_more_than_12_months_late':np.float32, 
# 'application_underwriting_score':np.float32,
# 'no_of_premiums_paid':np.float32, 
# 'sourcing_channel':np.str, 
# 'residence_area_type':np.str,
# 'premium':np.int}

Test_dtype = {'id':np.str, 
              'premium':np.int, 
              'renewal':np.float32,
              'incentives':np.float32
             }



## Loading dataset

In [4]:
# Loading dataset
tst_origin = pd.read_csv(Test_file, dtype=Test_dtype, na_values='')

display(tst_origin.head())

#tst_data = tst_origin[['id', 'premium']].copy()
tst_data = tst_origin

#Check the shape of each dataset
print(tst_data.shape)

,id,premium,renewal,improvement,incentives
0,649,3300,0.987610,0.0,0.0
1,81136,11700,0.977320,0.0,0.0
2,70762,11700,0.914971,0.0,0.0
3,53935,5400,0.965756,0.0,0.0
4,15476,9600,0.950142,0.0,0.0


(34224, 5)


### Arbitrarily put renewal probabilities for each policy

In [5]:
#pd_renewal = pd.DataFrame(np.random.rand(len(tst_data),1), columns=['renewal'])
#pd_incentives = pd.DataFrame(np.zeros((len(tst_data), 1)), columns=['incentives'], dtype=np.float32)
#pd_improve = pd.DataFrame(np.zeros((len(tst_data), 1)), columns=['improvement'], dtype=np.float32)
#tst_data2 = pd.concat([tst_data, pd_renewal, pd_improve, pd_incentives], axis=1)

tst_data2 = tst_data
display(tst_data2.head())
tst_data2.shape

,id,premium,renewal,improvement,incentives
0,649,3300,0.987610,0.0,0.0
1,81136,11700,0.977320,0.0,0.0
2,70762,11700,0.914971,0.0,0.0
3,53935,5400,0.965756,0.0,0.0
4,15476,9600,0.950142,0.0,0.0


(34224, 5)

### Build equations

In [6]:
import math

def inc_to_eff(inc):
    return 10*(1-np.exp(-1.0*inc/400.0))

def eff_to_imp(eff):
    return 20*(1-np.exp(-1.0*eff/5.0))/100

# def derivative(inc, delta):
#     y1 = eff_to_imp(inc_to_eff(inc))
#     y2 = eff_to_imp(inc_to_eff(inc+delta))
#     return (y2-y1)/delta

def rev_policy(renewal, premium, inc):
    #print('--', renewal, eff_to_imp(inc_to_eff(inc)), premium, inc )
    return (renewal + renewal*eff_to_imp(inc_to_eff(inc))) * premium - inc

def derivatives(renewal, premium, inc, delta=1):
    y1 = rev_policy(renewal, premium, inc)
    y2 = rev_policy(renewal, premium, inc+delta)
    return (y2-y1)/delta

def total_net_rev(dataset):
    return dataset['']


In [4]:
print(inc_to_eff(1000))
print(eff_to_imp(10))
print(eff_to_imp(inc_to_eff(100)))
print(derivatives(0.5, 10000, 100, 1))
print(derivatives(0.5, 1000, 100, 1))
print(derivatives(0.8, 10000, 100, 1))
print(derivatives(0.5, 10000, 10, 1))


9.179150013761012
0.17293294335267745
0.0715012813499204
1.4938951338181141
-0.7506104866181431
2.990232214107891
3.624525429319874


### Ok, Let's find optimal solution!!!!

In [7]:
import time

#unit_inc = 100
learning_rate = 0.8
delta = 1
MAX_ITER = 100000
t1 = time.time()

itr = 0
prev_total_net_rev = 0
total_net_rev = 0

while True:
    if itr > 2  and total_net_rev-prev_total_net_rev < 10:
        print('Total net revenue is below 1!')
        break
    if itr > MAX_ITER:
        print('Maximum iteration reached.')
        break
    result = derivatives(tst_data2['renewal'], tst_data2['premium'], tst_data2['incentives'], delta)
    #print(result.head())
    cand_idx, deriv = result.idxmax(), result.max()
    #data.loc[data.bidder == 'parakeet2004', 'bidderrate'] = 100
    tst_data2.loc[cand_idx, 'incentives'] = tst_data2.loc[cand_idx, 'incentives'] + learning_rate * deriv
    tst_data2.loc[cand_idx, 'improvement'] = eff_to_imp(inc_to_eff(tst_data2.loc[cand_idx, 'incentives']))
    
    prev_total_net_rev = total_net_rev
    total_net_rev = (tst_data2['renewal'] + tst_data2['renewal']*tst_data2['improvement']) * tst_data2['premium'] - tst_data2['incentives']
    total_net_rev = np.sum(total_net_rev)
    
    if itr%100 == 0:
        print('%d\t%d\t%13.3f\t%13.3f\t%13.3f' %(itr, cand_idx, deriv, total_net_rev, total_net_rev-prev_total_net_rev))
    
    itr += 1

print(round(time.time() - t1), 'seconds.')
    
#np.exp(1/tst_data2['premium'])

0	29686	       58.768	349939805.064	349939805.064
100	32098	       57.466	350168096.379	     2244.986
200	19937	       55.112	350383622.554	     2078.032
300	15010	       47.905	350570330.002	     1601.297
400	3423	       43.333	350715957.222	     1326.831
500	4705	       41.271	350840644.518	     1220.729
600	26446	       40.839	350961436.404	     1196.531
700	26388	       39.934	351078133.665	     1146.453
800	9725	       38.960	351189111.732	     1093.747
900	20429	       37.215	351293501.576	      995.480
1000	10463	       35.981	351391118.826	      933.789
1100	18783	       34.905	351481669.412	      881.415
1200	8840	       33.664	351567889.367	      827.126
1300	29686	       33.006	351648459.547	      801.370
1400	30413	       32.763	351727518.264	      789.994
1500	25683	       32.433	351805643.426	      766.327
1600	2113	       32.068	351881845.961	      749.965
1700	14493	       31.522	351956348.505	      725.768
1800	28015	       31.038	352027714.958	      704.621
1900	670	 

15500	4061	       16.859	356799538.384	      216.802
15600	16590	       16.813	356821238.648	      216.235
15700	9884	       16.769	356842819.943	      215.121
15800	11165	       16.737	356864296.106	      214.661
15900	19522	       16.710	356885680.033	      213.072
16000	4885	       16.690	356906988.456	      212.580
16100	8658	       16.668	356928242.764	      211.793
16200	7857	       16.653	356949454.745	      211.654
16300	30016	       16.636	356970617.952	      210.986
16400	4324	       16.621	356991745.774	      211.739
16500	26296	       16.605	357012828.985	      210.234
16600	4050	       16.594	357033887.831	      210.741
16700	8463	       16.576	357054898.054	      210.312
16800	15992	       16.561	357075865.746	      209.382
16900	15204	       16.541	357096793.554	      208.638
17000	18963	       16.523	357117664.965	      208.712
17100	30241	       16.508	357138500.633	      208.061
17200	3392	       16.490	357159291.003	      207.636
17300	26781	       16.468	357180038.8

30800	16065	       13.583	359492266.383	      142.412
30900	23583	       13.573	359506489.739	      142.504
31000	18078	       13.564	359520697.683	      141.882
31100	31357	       13.553	359534880.444	      141.781
31200	9062	       13.542	359549042.492	      141.552
31300	20880	       13.532	359563190.456	      141.361
31400	5092	       13.522	359577312.044	      140.673
31500	28681	       13.512	359591423.505	      141.242
31600	18765	       13.503	359605512.133	      140.296
31700	20202	       13.493	359619583.545	      140.553
31800	19033	       13.482	359633632.915	      140.198
31900	30464	       13.472	359647654.433	      140.418
32000	22857	       13.462	359661661.620	      139.920
32100	19041	       13.452	359675650.245	      139.461
32200	5956	       13.441	359689608.614	      139.768
32300	2081	       13.430	359703553.963	      139.701
32400	14371	       13.418	359717482.362	      138.771
32500	14080	       13.405	359731383.736	      139.205
32600	1432	       13.390	3597452

46100	3877	       11.835	361393087.233	      109.027
46200	7817	       11.812	361403941.115	      108.822
46300	29542	       11.794	361414764.368	      107.916
46400	8724	       11.772	361425548.285	      107.434
46500	15388	       11.751	361436285.733	      106.925
46600	531	       11.736	361446984.806	      106.862
46700	17701	       11.722	361457665.267	      106.883
46800	5538	       11.711	361468324.527	      106.686
46900	6037	       11.704	361478972.201	      106.650
47000	10173	       11.696	361489608.200	      106.159
47100	23523	       11.689	361500227.065	      106.023
47200	12458	       11.681	361510833.623	      105.890
47300	2004	       11.675	361521428.300	      105.771
47400	8177	       11.669	361532006.414	      105.663
47500	991	       11.662	361542573.869	      105.543
47600	16728	       11.655	361553130.430	      105.425
47700	27647	       11.649	361563674.112	      105.243
47800	12039	       11.644	361574207.638	      105.308
47900	33397	       11.638	361584728.966

61400	26483	       10.685	362895136.350	       89.115
61500	15419	       10.680	362904014.355	       88.587
61600	12816	       10.675	362912884.270	       88.505
61700	33736	       10.670	362921748.873	       89.002
61800	24618	       10.664	362930604.393	       88.326
61900	12907	       10.660	362939450.375	       88.249
62000	12466	       10.654	362948284.335	       88.823
62100	7068	       10.647	362957109.026	       88.299
62200	16548	       10.642	362965922.946	       88.102
62300	1898	       10.635	362974727.146	       88.105
62400	13579	       10.628	362983520.668	       87.740
62500	26217	       10.621	362992303.912	       87.543
62600	29332	       10.615	363001071.768	       87.885
62700	20545	       10.609	363009835.136	       87.423
62800	21605	       10.603	363018586.165	       87.424
62900	23853	       10.597	363027332.668	       87.313
63000	18407	       10.590	363036066.523	       87.623
63100	28827	       10.585	363044792.646	       87.034
63200	16848	       10.579	3630

76700	3557	        9.870	364150743.414	       75.853
76800	31781	        9.865	364158339.439	       76.160
76900	23494	        9.860	364165930.077	       75.704
77000	124	        9.855	364173513.380	       75.536
77100	2344	        9.850	364181087.642	       75.747
77200	24125	        9.844	364188655.141	       75.471
77300	2554	        9.839	364196212.164	       75.769
77400	21777	        9.834	364203760.927	       75.247
77500	2622	        9.829	364211299.622	       75.610
77600	10614	        9.824	364218831.110	       75.227
77700	12873	        9.820	364226359.568	       75.097
77800	33588	        9.815	364233880.345	       74.963
77900	23531	        9.810	364241392.556	       74.954
78000	29449	        9.805	364248897.077	       74.945
78100	16110	        9.801	364256395.310	       74.779
78200	24259	        9.795	364263885.706	       75.056
78300	29796	        9.791	364271364.556	       74.600
78400	17374	        9.787	364278835.668	       74.601
78500	9202	        9.782	364286301

92000	7214	        9.179	365234893.890	       65.934
92100	19106	        9.175	365241476.418	       65.848
92200	7196	        9.172	365248057.471	       65.800
92300	6271	        9.168	365254632.063	       65.556
92400	5081	        9.165	365261202.559	       65.564
92500	23762	        9.162	365267769.220	       66.013
92600	2266	        9.159	365274329.920	       65.430
92700	19543	        9.156	365280885.828	       65.389
92800	27542	        9.153	365287439.321	       65.539
92900	22309	        9.150	365293987.286	       65.500
93000	11678	        9.148	365300533.340	       65.804
93100	4107	        9.145	365307073.103	       65.225
93200	25835	        9.142	365313606.762	       65.380
93300	15536	        9.139	365320140.419	       65.522
93400	17425	        9.136	365326667.776	       65.302
93500	26685	        9.134	365333194.503	       65.044
93600	33871	        9.131	365339714.241	       65.363
93700	14579	        9.127	365346228.419	       64.955
93800	25491	        9.124	36535274

## Save output file for submission

In [8]:
display(tst_data2)

,id,premium,renewal,improvement,incentives
0,649,3300,0.987610,0.000000,0.000000
1,81136,11700,0.977320,0.021461,23.372049
2,70762,11700,0.914971,0.014182,14.986882
3,53935,5400,0.965756,0.000000,0.000000
4,15476,9600,0.950142,0.000000,0.000000
5,64797,11700,0.973035,0.021372,23.265966
6,67412,3300,0.582427,0.000000,0.000000
7,44241,5400,0.864581,0.000000,0.000000
8,5069,13800,0.980968,0.037096,43.291046
9,16615,28500,0.985644,0.095584,157.197906


In [10]:
tst_data2[['id', 'renewal', 'incentives']].to_csv('./Dataset/output_submission.csv', index=False)